### Оптимизаторы, параметры и модули

$\hat{y} = \sigma(w^T x + b)$

$\sigma(t) = \frac{1}{1 + \exp(-t)}$

$\text{CE}(y, \hat{y}) = -y \cdot \log \hat{y} - (1 - y) \log (1 - \hat{y})$

<img src="https://pytorch.org/tutorials/_images/comp-graph.png" style="background:white" width="600"/>

In [1]:
import torch

In [2]:
torch.manual_seed(42)
n_samples = 16
n_features = 5
x = torch.randn(n_samples, n_features)  # входной тензор
y = torch.randint(2, size=(n_samples, 1)).float()  # выходной тензор
w = torch.randn(
    n_features, 1, requires_grad=True
)  # параметр, хотим обновлять градиентным спуском
b = torch.randn(1, requires_grad=True)  # параметр, хотим обновлять градиентным спуском

In [3]:
loss = torch.nn.functional.binary_cross_entropy_with_logits(x @ w + b, y)
print(f"Начальное значение ошибки: {loss:.4f}")

Начальное значение ошибки: 0.8950


#### 1. Добавляем оптимизатор

In [4]:
optimizer = torch.optim.SGD([w, b], lr=0.1)

In [5]:
n_iter = 1000
step = 0.1
for i in range(n_iter):
    z = x @ w + b
    loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)
    # посчитаем производные
    loss.backward()
    # обновим значения параметров
    optimizer.step()
    # обнулим градиенты, мы не хотим их накапливать в данном случае
    optimizer.zero_grad()

# посмотрим, уменьшилось ли значение ошибки
print(loss)

tensor(0.3861, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)


#### 2. Прячем параметры модели внутрь `torch.nn.Module`

In [6]:
class LogReg(torch.nn.Module):
    def __init__(self, in_features: int, n_classes: int) -> None:
        super().__init__()
        self.weight = torch.nn.Parameter(torch.randn(in_features, n_classes))
        self.bias = torch.nn.Parameter(torch.randn(n_classes))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return x @ self.weight + self.bias

In [7]:
torch.manual_seed(42)
n_samples = 16
n_features = 5
x = torch.randn(n_samples, n_features)  # входной тензор
y = torch.randint(2, size=(n_samples, 1)).float()  # выходной тензор

logreg = LogReg(n_features, 1)
optimizer = torch.optim.SGD(logreg.parameters(), lr=0.1)

In [8]:
n_iter = 1000
step = 0.1
for i in range(n_iter):
    z = logreg.forward(x)
    loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)
    # посчитаем производные
    loss.backward()
    # обновим значения параметров
    optimizer.step()
    # обнулим градиенты, мы не хотим их накапливать в данном случае
    optimizer.zero_grad()

# посмотрим, уменьшилось ли значение ошибки
print(loss)

tensor(0.3861, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)


#### 3. Как учить на GPU

In [ ]:
accelerator = "cuda"  # 'cuda' для NVIDIA GPU или 'mps' для Macbook серии M
print(x.device)
x = x.to(device=accelerator)
print(x.device)

cpu
mps:0


In [10]:
x = x.to(device=accelerator)
y = y.to(device=accelerator)
logreg.to(device=accelerator)

for i in range(n_iter):
    z = logreg.forward(x)
    loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)
    # посчитаем производные
    loss.backward()
    # обновим значения параметров
    optimizer.step()
    # обнулим градиенты, мы не хотим их накапливать в данном случае
    optimizer.zero_grad()

# посмотрим, уменьшилось ли значение ошибки
print(loss)

tensor(0.3833, device='mps:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)


#### 4. Как сохранять и загружать веса модели

Способ 1 (предпочтительный): сохраняем только веса

In [11]:
# сохраняем состояние модуля
torch.save(logreg.state_dict(), "logreg.pt")

# загружаем
new_model = LogReg(n_features, 1)
new_model.load_state_dict(torch.load("logreg.pt", weights_only=True))
z = new_model.forward(x.to(device="cpu"))
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y.to(device="cpu"))
print(loss)

tensor(0.3833, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)


Способ 2: сохраняем вместе со структурой класса (использует `pickle`)

In [12]:
# сохраняем состояние модуля
torch.save(logreg, "logreg.pt")

# загружаем
new_model = torch.load("logreg.pt", weights_only=False)
z = new_model.forward(x)
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)
print(loss)

tensor(0.3833, device='mps:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
